In [1]:
# Some las data are so large to read them and voxelize them.
# here I show a way to do it...which is memory frienldy.
# In 'pcd_read_write.ipynb' notebook, I showed it how to do it with pcd files
# but it did not work that way, since memory didn't matter at that notebook case.

In [28]:
import laspy
import open3d as o3d
import numpy as np

In [29]:
from pathlib import Path

In [35]:
file_addr = Path("./lidar data/sample.las")

In [36]:
# file_addr = '/home/sobhan/Downloads/lidar data/InterpineData/LIR9_UNC_0_0 - Cloud-cleaned.las'

In [37]:
# laspy.read(file_addr)

In [38]:
las = laspy.read(str(file_addr))

FileNotFoundError: [Errno 2] No such file or directory: 'lidar data/sample.las'

In [26]:
las.header

<LasHeader(1.2, <PointFormat(1, 0 bytes of extra dims)>)>

In [9]:
las.header.point_format


<PointFormat(1, 0 bytes of extra dims)>

In [10]:

las.header.point_count



4180801

In [11]:
las.vlrs



[<GeoKeyDirectoryVlr(25 geo_keys)>, <GeoDoubleParamsVlr([c_double(43.4202777777778), c_double(607000.003), c_double(0.023), c_double(-88.8166666666667), c_double(1.0000333645), c_double(0.30480060960121924), c_double(6378137.0), c_double(298.2572221010042), c_double(0.0), c_double(0.017453292519943278)])>, <GeoAsciiParamsVlr(['NAD_1983_2011_WISCRS_Waupaca_Feet|projection: Transverse Mercator|PCS Name = NAD_1983_2011_WISCRS_Waupaca_Feet|NAVD88 - Geoid12B (Feet)|GCS Name = GCS_NAD_1983_2011|Datum = NAD_1983_2011|Ellipsoid = GRS_1980|Primem = Greenwich|', ''])>, <VLR(user_id: 'NIIRS10', record_id: '4', data len: 10)>, <VLR(user_id: 'NIIRS10', record_id: '1', data len: 26)>]

In [12]:
# since the code format is 1.1 it is categorized as sth that must be read from
# https://pro.arcgis.com/en/pro-app/3.0/help/data/las-dataset/storing-lidar-data.htm



list(las.point_format.dimension_names)



['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'synthetic',
 'key_point',
 'withheld',
 'scan_angle_rank',
 'user_data',
 'point_source_id',
 'gps_time']

In [13]:
las.X#, las.Y,las.Z

array([54450247, 54450263, 54450263, ..., 54899792, 54899791, 54899932],
      dtype=int32)

In [14]:

# 1 = Unassigned
# 2 = Ground
# 5 = High Vegetation
# 6 = Building
las.classification

<SubFieldView([2 2 2 ... 2 2 2])>

In [15]:
las.X.shape

(4180801,)

In [16]:
las.X.max(), las.X.min()

(54899999, 54450000)

In [17]:
coef_cubicize = (las.X.max()-las.X.min())/255
coef_cubicize

1764.7019607843138

In [18]:
(las.X-las.X.min())/255

array([9.68627451e-01, 1.03137255e+00, 1.03137255e+00, ...,
       1.76389020e+03, 1.76388627e+03, 1.76443922e+03])

In [19]:
def slotting(indices, cube_size):
    '''quite like cube_voxel(indices, cube_size) function in ./pcd_read_write.ipynb  notebook
    but this one is only works on on 1d np.ndarray not  a 3d np.ndarray. You can also use that function
    but here just a verion of that...
    '''
    cube_size = cube_size-1 #(ex. 256-->max will be 255)
    coef_cubicize = (indices.max(axis=0)-indices.min(axis=0))/cube_size

    return ((indices-indices.min(axis=0))//coef_cubicize).astype(int)
#     cube_voxels = cube_voxels.astype(int)
#     return cube_voxels

In [20]:
slotting(las.X, 64)

array([ 0,  0,  0, ..., 62, 62, 62])

In [21]:

# If I use 3d ndarry I would run out of mem, so I use for loop for values at the cost of losing time
# but saving mem, Since .las file here is supposed to be really big....


vox = np.zeros((64,64,64))
coef_x = (las.X.max()-las.X.min())/63
min_x  = las.X.min()

coef_y = (las.Y.max()-las.Y.min())/63
min_y  = las.Y.min()

coef_z = (las.Z.max()-las.Z.min())/63
min_z  = las.Z.min()

for index in range(las.X.shape[0]):
    ix = ((las.X[index]-min_x)//coef_x).astype(int)
    iy = ((las.Y[index]-min_y)//coef_y).astype(int)
    iz = ((las.Z[index]-min_z)//coef_z).astype(int)
    vox[ix, iy, iz] = 1
#     pass

KeyboardInterrupt: 

In [ ]:
vox.shape

In [39]:
from utils import save_mat, save_xyz

In [40]:
save_xyz(vox, 'output-complete-cleaned-Brendan.xyz')

voxel cube data is stored at  output-complete-cleaned-Brendan.xyz


In [33]:
save_mat(vox, 'output-complete-cleaned.mat')

In [22]:
# above code might be double and you need to convert it to int
# in following junks you can convert it to int....Just in case

In [11]:
import scipy.io
mat = scipy.io.loadmat('output-complete.mat')

In [12]:
vox = mat['vox'].astype(int)

In [13]:
save_mat(vox, 'output-complete1.mat')

In [15]:
save_xyz(vox, 'output-complete1.xyz')